# 1、简介

基于卷积神经网络的深度学习模型在各种计算机视觉任务中都有优秀的表现。要评估一个模型的好坏，我们不仅仅要看模型的效果如何，还要考虑模型的计算代价。

在实际工业界的应用中，特别是边缘计算领域，受限于应用需求，我们常常需要在计算性能受限的硬件平台运行神经网络，如交通应用中的行人检测就是典型的高实时性需求场景，在输入数据后，我们需要网络能够迅速的给出响应。这时就需要一个能够在一定程度上表征模型复杂程度的参数，用于模型的选择和优化。

![](https://ai-studio-static-online.cdn.bcebos.com/a4af146deda743bba2b61d2cd6e2429e9634b4766424430a942f8b9e1e02a665)


**FLOPs（FLoating point OPerations）**
浮点运算操作数就可以用来衡量算法/模型的复杂度，该参数代表网络完成计算所需要的浮点运算次数。
   
       注意与FLOPS(floating point operations per second)区分，后者是表示每秒的浮点运算次数，一般用于表征硬件的计算能力。

需要注意的是，**使用FLOPs只能大致地对网络计算代价进行估计**，实际的网络运行速率还受到很多因素影响，如内存带宽，线程，程序优化程度等等。

# 2、普通卷积层的计算量
![](https://ai-studio-static-online.cdn.bcebos.com/f8bd90fa532c400aac0f73a97886660295d0becb089844c78a66daff189ed5a2)

## 2.1、卷积层的参数量
根据卷积操作的定义，首先假设输入通道为*Cin*，输出的feature map通道数为*Cout*，每个输入通道的计算过程如图所示，默认stride=1，无空洞。目前常用的卷积核一般为正方形，图中的卷积核边长为*K*=3，则每个卷积核的参数量为**（3x3+1）=10**，+1是考虑偏置的情况，如果不考虑偏置则没有+1项。再考虑输入通道和输出通道数，则一般的参数量计算公式为：

![](https://ai-studio-static-online.cdn.bcebos.com/6bb52c989ab142189221ad6f9fc8286ceed80db2d2124082996ac9aa175891d8)

## 2.2、卷积层的FLOPs
计算卷积层的FLOPs需要结合卷积的原理来推导，要得到feature map第一个值12需要的计算为**（3x3）** 次乘法和**（3x3-1）** 次加法，考虑偏置项需要额外一次加法，总共需要 **（3x3+3x3-1+1）= 18** 次计算，图中的feature map共9个值，因此实际需要计算 **（9x18）= 162** 次。考虑输入通道数和输出通道数，则一般的FLOPs计算公式为：

![](https://ai-studio-static-online.cdn.bcebos.com/054e4be353b7498f9275c694f5ce4a79cca09c9958204f9ca8f1e8679b3d8696)

其中*H，W*为feature map的尺寸



# 3、全连接层的计算量


全连接层在整个卷积神经网络中起到“分类器”的作用。全连接层实际是在进行向量和矩阵的乘法运算，输入一个*I*维向量与*IxJ*维矩阵相乘得到输出的*J*维向量。


## 3.1、全连接层的参数量
它的参数计算比较简单，对于输入为*I*维，输出为*J*维的全连接层，其参数量计算公式为：

![](https://ai-studio-static-online.cdn.bcebos.com/79b914f600af40ab97422955d1ca499760fda1ff5afd4d849aba386622ea1ef8)

其中+1为偏置项。

## 3.2、全连接层的FLOPs
类似的，在考虑偏置的情况下，FLOPs的计算公式为：

![](https://ai-studio-static-online.cdn.bcebos.com/543e0ad53a7a4736be9ce93cb328a818f3867e40f08e45b48ddd573022cd64c2)

**值得注意的是，当fc层输入输出维数较大时，如下面的AlexNet，fc层的参数量会远多于卷积层**。



In [25]:
# 以AlexNet为例计算一下网络的计算量
import paddle
import paddle.nn as nn

# 网络结构的参数
in_channels=[3,96,256,384,384]
out_channels=[96,256,384,384,256]
kernel_size=[11,5,3,3,3]
stride=[4,1,1,1,1]
padding=[0,2,1,1,1]

# 计算每层卷积之后feature_map的size
def compute_out_size(in_size,kernel_size,stride,padding):
    out_size = (in_size + 2*padding-kernel_size)/ stride + 1
    return out_size

# 计算卷积层的相关数据
def compute_conv_parm(in_channels,out_channels,kernel_size,stride,padding,size=227):
    size = [size]
    parm = []
    flops = []
    for i in range(len(in_channels)):
        size.append(compute_out_size(size[-1],kernel_size[i],stride[i],padding[i]))
        parm.append(((kernel_size[i]**2)*in_channels[i] + 1)*out_channels[i])
        flops.append(2*(kernel_size[i]**2)*size[-1]*size[-1]*in_channels[i]*out_channels[i])
    return parm,flops

#计算fc层的相关数据
def compute_fc_parm(in_features,out_features):
    parm = []
    flops =[] 
    for i in range (len(in_features)):
        parm.append((in_features[i]+1)*out_features[i])
        flops.append((2*in_features[i]-1)*out_features[i])
    return parm,flops

#网络结构
class AlexNet(nn.Layer):
    def __init__(self,in_channels,out_channels,kernel_size,stride,padding):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2D(in_channels=in_channels[0], 
                      out_channels=out_channels[0], 
                      kernel_size=kernel_size[0], 
                      stride=stride[0], 
                      padding=padding[0]),
            nn.ReLU(),
            nn.MaxPool2D(3, 2))
        self.conv2 = nn.Sequential(
            nn.Conv2D(in_channels=in_channels[1], 
                      out_channels=out_channels[1], 
                      kernel_size=kernel_size[1], 
                      stride=stride[1], 
                      padding=padding[1]),
            nn.ReLU(),
            nn.MaxPool2D(3, 2))
        self.conv3 = nn.Sequential(
            nn.Conv2D(in_channels=in_channels[2], 
                      out_channels=out_channels[2], 
                      kernel_size=kernel_size[2], 
                      stride=stride[2], 
                      padding=padding[2]),
            nn.ReLU())
        self.conv4 = nn.Sequential(
            nn.Conv2D(in_channels=in_channels[3], 
                      out_channels=out_channels[3], 
                      kernel_size=kernel_size[3], 
                      stride=stride[3], 
                      padding=padding[3]),
            nn.ReLU())
        self.conv5 = nn.Sequential(
                nn.Conv2D(in_channels=in_channels[4], 
                      out_channels=out_channels[4], 
                      kernel_size=kernel_size[4], 
                      stride=stride[4], 
                      padding=padding[4]),
            nn.ReLU(),
            nn.MaxPool2D(3,2))
        self.fc = nn.Sequential(
            nn.Linear(in_features= out_channels[4]*6*6,out_features=4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(in_features=4096,out_features=4096),
            nn.ReLU(),
            nn.Linear(in_features=4096,out_features=1000))
        self.flatten=nn.Flatten()

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x


if __name__ == '__main__':
    net = AlexNet(in_channels,out_channels,kernel_size,stride,padding)
    # input_tensor = paddle.tensor.rand([4,3,227,227])
    # output = net(input_tensor)
    parm_conv,flops_conv = compute_conv_parm(in_channels,out_channels,kernel_size,stride,padding)
    parm_fc,flops_fc = compute_fc_parm([256*6*6,4096,4096],[4096,4096,1000])
    print("每个卷积层的参数量/flops:",parm_conv,flops_conv)
    print("每个fc层的参数量/flops:",parm_fc,flops_fc)
    paddle.summary(net=net,input_size=(4,3,227,227))

每个卷积层的参数量/flops: [34944, 614656, 885120, 1327488, 884992] [210830400.0, 3717120000.0, 5352652800.0, 8028979200.0, 5352652800.0]
每个fc层的参数量/flops: [37752832, 16781312, 4097000] [75493376, 33550336, 8191000]
---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Conv2D-41     [[4, 3, 227, 227]]    [4, 96, 55, 55]        34,944     
    ReLU-57      [[4, 96, 55, 55]]     [4, 96, 55, 55]           0       
 MaxPool2D-25    [[4, 96, 55, 55]]     [4, 96, 27, 27]           0       
   Conv2D-42     [[4, 96, 27, 27]]     [4, 256, 27, 27]       614,656    
    ReLU-58      [[4, 256, 27, 27]]    [4, 256, 27, 27]          0       
 MaxPool2D-26    [[4, 256, 27, 27]]    [4, 256, 13, 13]          0       
   Conv2D-43     [[4, 256, 13, 13]]    [4, 384, 13, 13]       885,120    
    ReLU-59      [[4, 384, 13, 13]]    [4, 384, 13, 13]          0       
   Conv2D-44     [[4, 384, 13, 13]]    [4, 384, 13, 1

> 推荐使用paddle.summary（）这个api，参数是网络实例和输入的shape，它可以直接打印网络每一层的输入和输出shape以及各种参数信息。

# 4、深度可分离卷积的计算量

论文地址：[MobileNets: Efficient Convolutional Neural Networks for Mobile Vision
Applications](http://arxiv.org/abs/1704.04861)

这里再补充一个能有效地降低网络参数量的卷积方式——**深度可分离卷积**（depthwise-separable convlution）

> 论文原图：![](https://ai-studio-static-online.cdn.bcebos.com/6a12fcc808c14ceb847c329da094aa1e510aa1682f8d48c6947ef2ad43094cfd)

深度可分离卷积将卷积操作分成两步：
## 4.1、深度卷积

第一步是**深度卷积**，可以理解为逐通道卷积，保持通道数不变，每个通道仅对应一个卷积核，深度卷积的计算量为：

![](https://ai-studio-static-online.cdn.bcebos.com/30e1d9dcd5514d54991d3dd46ff5fc529c419d1441a24329aaf1601c8d896dac)

经过深度卷积，我们得到了*Cin*维的Feature_map。此时还需要使用一个1x1的卷积来整合通道间的信息。

## 4.2、逐点卷积

第二步是**逐点卷积**，逐点卷积的卷积核大小始终保持为1，通过逐点卷积仅改变了feature map的通道数。逐点卷积的计算量为：

![](https://ai-studio-static-online.cdn.bcebos.com/8ecd08634d3c43e8a2e4de8a1cfa4e5be39fab017c5540cbb51abebfdba4f818)

总的FLOPs为：

![](https://ai-studio-static-online.cdn.bcebos.com/5f1b839c4b3d4b91a850b0af99fd5a0e84dd4ee9270f4416a9a35b7a358e7777)

与普通卷积的计算量相除，即可得到深度可分离卷积的计算量约为普通卷积的***1/K^2***（忽略***1/Cout***）

使用深度可分离卷积大大降低了网络的计算代价，卷积核的尺寸越大，深度可分离卷积能够降低的复杂度就越多。

    在代码中，深度可分离卷积一般是通过分组卷积实现，令group=Cin，实现每个通道单独卷积。之后再加上一个1x1的卷积，调整feature map的通道数到Cout即可。

基于深度可分离卷积的网络如mobilenet系列已经在边缘计算领域得到了很好的应用。

In [26]:
class DepthwiseSeparable(nn.Layer):  # 深度可分离卷积
    def __init__(self, in_channels, out_channels1, out_channels2, num_groups, stride):
        super(DepthwiseSeparable, self).__init__()

        # 深度卷积
        self.depthwiseConv = ConvBnLayer(in_channels=in_channels,
                                         out_channels=int(out_channels1),
                                         num_groups=int(num_groups),
                                         kernel_size=3,
                                         stride=stride,
                                         padding=1)
        
        # 逐点卷积
        self.pointwiseConv = ConvBnLayer(in_channels=int(num_groups),
                                         out_channels=int(out_channels2),
                                         kernel_size=1,
                                         stride=1,
                                         padding=0)

    def forward(self, x):
        x = self.depthwiseConv(x)
        x = self.pointwiseConv(x)
        return x

if __name__ == "__main__":
    in_channels=[96]   #网络结构参数
    out_channels=[256]
    kernel_size=[5]
    stride=[1]
    padding=[2]
    DC_parm_conv, DC_flops_conv = compute_conv_parm(in_channels,[1],kernel_size,stride,padding,size=27)
    PC_parm_conv, PC_flops_conv = compute_conv_parm(in_channels,out_channels,[1],stride,[0],size=27)
    DCS_parm_conv=DC_parm_conv[0]+PC_parm_conv[0]  # 两个卷积操作计算量的和
    DCS_flops_conv=DC_flops_conv[0]+PC_flops_conv[0]
    trad_parm_conv, trad_flops_conv = compute_conv_parm(in_channels,out_channels,kernel_size,stride,padding,size=27)   
    print("dsc的参数量/flops:",DCS_parm_conv,DCS_flops_conv)
    print("普通卷积层的参数量/flops:",trad_parm_conv[0],trad_flops_conv[0])


dsc的参数量/flops: 27233 39331008.0
普通卷积层的参数量/flops: 614656 895795200.0


# 5、总结

以上就是CNN中FLOPs和参数量的定义与计算推导，实际使用中，这些参数一般是作为论文中与其他网络进行对比时的一个数据和部署时对硬件配置要求的参考。一些网络优化操作也可以降低网络的FLOPs，比如对网络的权重进行量化等。


## 作者简介
> 作者：yuukiyuuki

> paddle初学者，coding水平有待提高，研究方向是目标跟踪和边缘计算，搬砖中。。。

> 我在AI Studio上获得白银等级，点亮2个徽章，来互关呀~https://aistudio.baidu.com/aistudio/personalcenter/thirdview/801662
